# Flot Plot Line Chart Example using AJAX

Creating a plot with FlotPlot is pretty simple. It can be done with either hard coding data into the JavaScript file or using something like AJAX to asynchronously load the data. This is great because it means the page will load first and not be waiting on the data. The following will be some rough notes describing the code and the things that I learned while getting this to work. 

To begin we need some data, in this case I knew I would eventually be attempting to plot large datasets so an AJAx solution might be best to expierement with. In oder to make a simple example, I created a *sinuid* using `numpy` then converted it to a Pandas DataFrame so that it would be more representative of the types of data sources that I am use to working with. Once converted to a DataFrame the data source needed to be made into a json data type so that it could be served up with Flask.

I used the `orient='records'` argument so that it would preserve the column names. I wanted to be able to use them in the JavaScript code to help make it a little bit easier to understand.

In [1]:
import numpy as np
import pandas as pd
import json

@example_site.route('/example/test.json')
def send_sinwave():
    N = 1024
    ix = np.arange(N)
    y = np.sin(2 * np.pi * ix / float(N / 3)) * 20 + 30
    x = range(0,N)
    data = dict(X= x, Y=y)
    data = pd.DataFrame(data)
    return data.to_json(orient='records')

json.loads(send_sinwave())[:5]

[{'X': 0, 'Y': 30.0},
 {'X': 1, 'Y': 30.3681345981},
 {'X': 2, 'Y': 30.7361444588},
 {'X': 3, 'Y': 31.103904887},
 {'X': 4, 'Y': 31.471291272}]

Now that I have a datasource created, it is time to get to work on making an HTML template to display the plot once it is generated. The below code is all that is needed to let the JavaScript know where we want the plot to be placed on our page.

In [ ]:
<div class="container-fluid bg-1 text-center">
    <div class="col-lg-12">
            <div class="panel panel-default">
        <div class="panel-body">
            <div class="flot-chart">
                <div class="flot-chart-content" id="flot-line-chart"></div>
            </div>
        </div>
    </div>
    </div>
</div>

The JavaScript is really where all the work is done. The code to create the plot follows. I am horrible at JavaScript and so I used D3 to manipulate the data but it can be done probably a million other ways, I just found this to be the easiest and most comprehendable by my weak Jarhead mind. 

So first thing is we need to get the data from our datasource to JavaScript. To do this we use AJAX. The function is pretty simple. We pass it the `url` of the datasource, tell it to not get data asyncronously and pass it a success function. The reason for the `async: false` is because without this the AJAX function would start downloading the data then move on to do other things. The data would still be loading miliseconds later and the things that are dependent on the data would be called and be freaking out. The success function simply stores the data in a variable called `plot_data`. 

In [ ]:
$.ajax({
    url: '/example/test.json',
    dataType: 'json',
    async: false,
    success: function(data) {
    plot_data = data
        }
    });

Now that we have the data we need to get it into formats that Flot Plot will be happy with and to get some axis information. To get the axis information I used D3 because I thought it was a little easier to read. I then loop through the `plot_data` object to create a multi-deminsional array for Flot Plot. 

In [ ]:
var yaxis = {"Min":d3.min(plot_data, function(d) { return d.Y; }),
             "Max":d3.max(plot_data, function(d) { return d.Y; })};
var xaxis = {"Min":d3.min(plot_data, function(d) { return d.X; }),
             "Max":d3.max(plot_data, function(d) { return d.X; })};

var data_to_plot = [[]];

for(key in plot_data)
    data_to_plot.push([plot_data[key].X, plot_data[key].Y]);

In [ ]:
//    actual plot stuff
plot()
function plot() {

    var options = {
        series: {
            lines: {
                show: true
            },
            points: {
                show: true
            }
        },
        grid: {
            hoverable: true //IMPORTANT! this is needed for tooltip to work
        },
        yaxis: {
            min: (yaxis.Min) - (50), // Just giving some room above and below the sinwave
            max: (yaxis.Max) + (50)
        },
        xaxis: {
            min: xaxis.Min,
            max: xaxis.Max
        },
        tooltip: true,
        tooltipOpts: {
            content: "'%s' of %x.1 is %y.4",
        }
    };
    var plotObj = $.plot($("#flot-line-chart"), [{
            data: data_to_plot,
            label: "A sinwave made in Python."
        }],
        options);
}
});

plot1 here

## Multiple Line Series

To make things more interesting I decided to plot multiple lines. At first things were abit confusing because I am horrible with JavaScript, but after a little playing around this too is very easy to achieve. For this example I obviously need an additionaly datasource. I genereted this data source a little differently and made some tweaks to the original datasource because I found this method to be slightly better downstream. Take note that if you attempt to just send a Python dictionary wrapped in `jsonify()` or any of its sister functions it will kick back with an error about not being serialized. To fix this problem you can just pass the `jsonify` function key names like a dictionary, something II had not realized originally.. 

In [ ]:
@example_site.route('/example/test.json')
def send_sinwave():
    N = 1024
    ix = np.arange(N)
    y = np.sin(2 * np.pi * ix / float(N / 3)) * 20 + 30
    x = range(0, N)
    data = pd.DataFrame({'X': x, 'Y':  y}).to_dict(orient='records')
    return jsonify(DataTest=data, label='TestLabel')

@example_site.route('/example/test2.json')
def send_sinwave2():
    N = 2000
    ix = np.arange(N)
    y = np.sin(-2 * np.pi * ix / float(N / 3)) * -2
    x = range(0, N)
    data = pd.DataFrame({'X': x, 'Y':  y}).to_dict(orient='records')
    return jsonify(DataTest=data, label='TestLabel2')

To get the data we just add a second AJAX call that gets a url passed to it corrisponding with our second data source, then save the data off to a variable that I am creatively calling `plot_data2`. 

In [ ]:
var plot_data;
var plot_data2;

$.ajax({
    url: '/example/test.json',
    dataType: 'json',
    async: false,
    success: function(data) {
    plot_data = data
        }
    });

$.ajax({
    url: '/example/test2.json',
    dataType: 'json',
    async: false,
    success: function(data) {
    plot_data = data
        }
    });

Once the data is in JavaScript if we wrap both datasets in list we can use D3 to get the axis min and maxes over all datasets in one simple call. 

In [ ]:
var both_data = [plot_data,plot_data2];

var minY = d3.min(both_data, function(pd) { return d3.min(pd.DataTest, function(dt) { return dt.Y; }); });
var maxY = d3.max(both_data, function(pd) { return d3.max(pd.DataTest, function(dt) { return dt.Y; }); });
var minX = d3.min(both_data, function(pd) { return d3.min(pd.DataTest, function(dt) { return dt.X; }); });
var maxX = d3.max(both_data, function(pd) { return d3.max(pd.DataTest, function(dt) { return dt.X; }); });

Next comes something I am not super fond of and could be avoided if I just passed matrices from Python instead of labeled dictionary/DataFrames. Maybe there is a trick to do this in JavaScript that I am not aware of at this time but for now the following seems to be required because Flot Plot requires its data in multidemensional arrays, not as objects. 

In [ ]:
var data_to_plot = [[]];
var data_to_plot2 = [[]];

for(key in plot_data.DataTest)
    data_to_plot.push([plot_data.DataTest[key].X, plot_data.DataTest[key].Y]);

for(key in plot_data2.DataTest)
    data_to_plot2.push([plot_data2.DataTest[key].X, plot_data2.DataTest[key].Y]);

The only change to the actual plotting code was in the plotObj variable. Just add an additional object that represents the additional series. Some observations about how I might improve on things. I could see passing the color information and maybe even other details about the series in the json datasource. 

In [ ]:
var plotObj = $.plot($("#flot-line-chart"), [{
        data: data_to_plot,
        label: plot_data.label,
        color: 'blue'
    },
    {
        data: data_to_plot2,
        label: plot_data2.label,
        color: 'red'
    }],
    options);
}

plot2 here